In [1]:
#frameworks needed
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
#from keras.preprocessing.text import one_hot, text_to_word_sequence
#from keras import Sequential, layers, Input
#from keras.layers import Dense, LSTM, Dropout, Embedding, Lambda
from tensorflow.keras.utils import text_dataset_from_directory
import datetime, os
import nltk
from nltk.stem import LancasterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer 
import pickle
from nltk import word_tokenize
from statistics import mode
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from keras.models import Model
from keras import models
from keras import backend as K
#from keras.preprocessing.sequence import pad_sequences
#from keras.preprocessing.text import Tokenizer 
from keras.utils import plot_model
from keras.layers import Input,LSTM,Embedding,Dense,Concatenate,Attention
from sklearn.model_selection import train_test_split
# from bs4 import BeautifulSoup

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jared\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jared\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jared\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [4]:
lyrics=pd.read_csv("./data/datasetsGen/5milSong/ds2.csv",usecols=["lyrics"])

In [5]:
stop_words=set(stopwords.words('english'))
contractions=pickle.load(open("./data/datasetsGen/contractions.pkl","rb"))['contractions']
stemm=LancasterStemmer()

In [6]:
import string
for s in range(0,len(lyrics.values)):
  lyrics.values[s] = re.sub("\n"," ",str(lyrics.values[s][0]))
  lyrics.values[s] = re.sub("\"","",str(lyrics.values[s][0]))
  lyrics.values[s] = re.sub("[\(\[].*?[\)\]]", "", str(lyrics.values[s][0]))

In [9]:
lyrics.values

array([[" Killa Cam, Killa Cam, Cam Killa Cam, Killa Cam Killa Cam, Cam Killa Cam, Killa Cam, Cam Killa Killa Killa Cam Killa Cam, Cam, Killa  Killa Cam, Killa Cam, Cam  Killa Cam, Killa Cam  Killa Cam, Cam  Killa Cam, Killa Cam, Cam  Killa Killa Killa Cam Killa Cam, Cam, Killa  Killa Cam , Killa Cam, Cam Killa Cam, Killa Cam Killa Cam, Cam  Killa Cam, Killa Cam, Cam Killa Killa Killa Cam Killa Cam, Cam, Killa  Killa Cam, Killa Cam, Cam Killa Cam, Killa Cam Killa Cam, Cam Killa Cam, Killa Cam, Cam Killa Killa Killa Cam  Killa Cam, Cam, Killa  With the goons I spar, stay in tune with ma  She like, Damn, this the realest since 'Kumbaya' Bomaye, Killa Cam, my Lord  Still the man with the pan, scrilla, fam, on board Now bitches, they want to neuter me, niggas, they want to tutor me The hooligan in Houlihan's, maneuvering's nothing new to me Doggy, I'm from the land of grind, pan-pan: gram or dime? Not toes or MC when I say hammer time Beef: I hammer mine, when I get my hands on nines If I 

In [7]:
dataS=[]
for i in range(4000000,len(lyrics.values)):
    words= str(lyrics.values[i])
    #contraction file to expand shortened words
    words= [contractions[w] if w in contractions else w for w in words.split(" ")]
    #words= [stemm.stem(w) for w in words if w not in stop_words]
    dataS.append(words)

In [8]:
dataF=[]
for i in dataS:
    if len(i)==0:
        continue
    words=""
    for j in range(0,len(i)):
        if len(i[j].split(" "))>1:
            tmp=i[j].split(" ")
            words=words+tmp[0]+" "+tmp[1]+" "
        else:
            words=words+i[j]+" "
    dataF.append(words)

In [9]:
dataF

['["Niggas broke, yea they been mad I live the life that you dream yea Gucci that is all\\u2005on\\u2005my durag Fendi that\'s\\u2005all on my bitch bag Burken that\'s\\u2005all on my bitch bag Designer that is all on my shoe\\u205ftag Prada\\u205fthat\'s\\u205fall on my\\u205fbitch yea Juicy that\'s\\u205fall on my bitch ass  Walk in the store, bitch I buy out the shelf Diamonds stay wet, bitch call me Michael Phelps Might blow an ounce, or my brains I cannot tell Everyone know that I am wild as hell Might go kurt Cobain Might go Hitler mane All of these niggas they fake All of these niggas they lame  Got a mcm, backpack, with the red bottoms on. FUCK it Call it peanut butter insiddddees Control yo bitch or she might get hit She on dick cause she heard that I am that guuuuuuy I just flex up on my ex, cannot have me back, cause she been triiied She say she fucked up, fuuc u the one, gold on my teeth, now watch me shine Smoking on gas, for the whole damn nation No green card, Dr told me

In [10]:
import string
for s in range(0,len(dataF)):
  dataF[s] = str(dataF[s]).translate(str.maketrans('', '', string.punctuation)).strip()

In [11]:
dataF

['Niggas broke yea they been mad I live the life that you dream yea Gucci that is allu2005onu2005my durag Fendi thatsu2005all on my bitch bag Burken thatsu2005all on my bitch bag Designer that is all on my shoeu205ftag Pradau205fthatsu205fall on myu205fbitch yea Juicy thatsu205fall on my bitch ass  Walk in the store bitch I buy out the shelf Diamonds stay wet bitch call me Michael Phelps Might blow an ounce or my brains I cannot tell Everyone know that I am wild as hell Might go kurt Cobain Might go Hitler mane All of these niggas they fake All of these niggas they lame  Got a mcm backpack with the red bottoms on FUCK it Call it peanut butter insiddddees Control yo bitch or she might get hit She on dick cause she heard that I am that guuuuuuy I just flex up on my ex cannot have me back cause she been triiied She say she fucked up fuuc u the one gold on my teeth now watch me shine Smoking on gas for the whole damn nation No green card Dr told me be patient Smoke more than snoop 3 pounds

In [12]:
for i in dataF:
    if i=="" or i==" ":
        dataF.remove(i)

In [13]:
from lingua import Language, LanguageDetectorBuilder
languages = [Language.ENGLISH,Language.FRENCH, Language.GERMAN, Language.SPANISH]
detector = LanguageDetectorBuilder.from_languages(*languages).with_minimum_relative_distance(0.9).build()
#language = detector.detect_language_of("languages are awesome")

In [14]:
from langdetect import detect
data=[]
i=0
for k in range(0,len(dataF)):
    try:
        lan=detector.detect_language_of(dataF[i])
    except Exception:
        continue
    else:
        if lan==Language.ENGLISH:
            tmp=""
            l=0
            for y in dataF[i].split(" "):
                tmp=tmp+" "+y
                l+=1
                if l>500:
                    break
            data.append(tmp)
    finally:
        i+=1

In [20]:
data

,Lyrics
0,Killa Cam Killa Cam Cam Killa Cam Killa Cam K...
1,Yeah hah yeah RocAFella We invite you to some...
2,Maybe cause I am eatin And these bastards fie...
3,Ugh Killa Baby Kanye this that 1970s Heron fl...
4,So they ask me Young boy What you gon do the ...
...,...
1471000,Quante scarpe comprerai Quante ne hai già con...
1471001,È questa la mia fortuna Che sono nato qua Ad ...
1471002,Mi sono rotta un braccio tenendoti la mano Il...
1471003,Adesso sono libera E se lo sono libero anche ...


In [15]:
data = pd.DataFrame(data, columns=['Lyrics'])

In [16]:
data.to_csv("./data/2milSongClean2.csv")